# Object Detection

1. Introduction to Object Detection
- Object detection is a key technique in computer vision that involves identifying and locating objects within an image or video. - - It goes a step beyond image classification by not only categorizing the objects but also pinpointing their positions. 
- This technology has significant applications in various fields, from surveillance systems that can detect suspicious activities to autonomous vehicles that need to recognize obstacles. 
- For instance, in retail, object detection helps in automated checkout systems by recognizing products without the need for barcodes.

### 2. Key Components of Object Detection
- Landmarks: Specific points within an object (like corners of the eyes or nose tip in face detection).
- Anchors: Predetermined boxes of various scales and aspect ratios that serve as reference points.
- Bounding Boxes: Rectangles that enclose objects, defined by coordinates (x, y, width, height).
- Grids: The image is divided into a grid, and object detection is performed in each grid cell.

### 3. Implementing Basic Object Detection
- We'll start by implementing a simple object detection system using a pre-trained model from TensorFlow's model zoo.

In [ ]:
pip install opencv-python

In [ ]:
pip install tensorflow-hub

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import tensorflow_hub as hub

# Load the model from TensorFlow Hub
model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v1/fpn_1024x1024/1")

# Function to run object detection
def detect_objects(image_np, model):
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = model(input_tensor)

    # Extract data
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # Detection classes and boxes
    detection_classes = detections['detection_classes'].astype(np.int64)
    detection_boxes = detections['detection_boxes']

    return detection_boxes, detection_classes

# Example usage
image_np = cv2.imread('path_to_your_image.jpg')
boxes, classes = detect_objects(image_np, model)

# Draw bounding boxes (example for visualization)
for box in boxes:
    y_min, x_min, y_max, x_max = box
    start_point = (int(x_min * image_np.shape[1]), int(y_min * image_np.shape[0]))
    end_point = (int(x_max * image_np.shape[1]), int(y_max * image_np.shape[0]))
    cv2.rectangle(image_np, start_point, end_point, (255, 0, 0), 2)

cv2.imshow('Object Detection', image_np)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 4. Improving Accuracy with Non-Max Suppression
- Non-Max Suppression (NMS) is used to eliminate redundant overlapping bounding boxes.

In [ ]:
def non_max_suppression(boxes, scores, threshold):
    assert len(boxes) == len(scores)
    # Initialize an empty list for selected boxes
    selected_boxes = []

    # Sort the boxes by scores in descending order
    sorted_indices = np.argsort(scores)[::-1]

    while len(sorted_indices) > 0:
        # Select the box with the highest score
        selected_box_index = sorted_indices[0]
        selected_boxes.append(boxes[selected_box_index])

        # Compute IoU of the selected box with the rest
        ious = np.array([iou(boxes[selected_box_index], box) for box in boxes[sorted_indices[1:]]])

        # Remove indices of boxes with IoU greater than the threshold
        sorted_indices = np.delete(sorted_indices, np.where(ious > threshold)[0] + 1)
        sorted_indices = np.delete(sorted_indices, 0)

    return np.array(selected_boxes)

# You'll need to implement the 'iou' function to calculate Intersection over Union

### 5. Intersection Over Union (IoU)
- IoU is a measure to evaluate how much two bounding boxes overlap.

In [ ]:
def iou(box1, box2):
    # Calculate intersection area
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Calculate union area
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - intersection_area

    return intersection_area / union_area